# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

from PIL import Image
import pickle

In [2]:
%matplotlib inline
%matplotlib tk

# Provided functions

In [3]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.axis('off')
    plt.imshow(im)
    print(f'Set at least {min_lines} lines to compute vanishing point')
    print(f'The delete and backspace keys act like right clicking')
    print(f'The enter key acts like middle clicking')
    while True:
        print('Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print(f'Need at least {min_lines} lines, you have {n} now')
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [4]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [5]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    Inputs:
        lines: np.ndarray of shape (3, N)
            where each column denotes the parameters of the line equation (i.e. coefficients in $ax + by + c = 0$)
    Returns:
        vp: np.ndarray of shape (3, )
            where the 2d vanishing point in homogeneous coordinate obtained by least-square solution among N lines
    """
    _, N = lines.shape

    A = np.empty((N, 3))
    for i in range(N):
        A[i, :] = lines[:, i]
    
    U, S, V = np.linalg.svd(A)
    vp = np.reshape(V[-1, :], (3,))
    
    vp /= vp[-1]

    return vp

In [6]:
def get_horizon_line(horizontal_vp0, horizontal_vp1):
    """
    Calculates the ground horizon line.
    Inputs:
        horizontal_vp0: np.ndarray of shape (3, )
            where one horizontal 2d vanishing point in homogeneous coordinate
        horizontal_vp1: np.ndarray of shape (3, )
            where the other horizontal 2d vanishing point in homogeneous coordinate
    Returns:
        horizontal_line: np.ndarray of shape (3, )
            where the coefficients of 2d horizontal line equation ax + by + c = 0 
            with a constraint that a**2 + b**2 = 1
    """
    # normalize 2d vanishing points 
    horizontal_vp0 /= horizontal_vp0[-1]
    horizontal_vp1 /= horizontal_vp1[-1]

    # construct ax + by + c = 0 where coefficients are not normalized
    x0, y0 = horizontal_vp0[:-1]
    x1, y1 = horizontal_vp1[:-1]
    horizontal_line = np.array([(y1 - y0), -(x1 - x0), -x0*y1 + x1*y0])
    
    # force ax + by + c = 0 where a**2 + b**2 = 1
    horizontal_line /= np.linalg.norm(horizontal_line[:-1])
    
    return horizontal_line

In [7]:
def plot_horizon_line(ax, im, horizontal_line, pt1=None, pt2=None):
    """
    Plots the horizon line.
    Inputs:
        ax: pyplot axis
        im: np.ndarray of shape (height, width, 3)
        horizontal_line: np.ndarray of shape (3, )
            where the coefficients of 2d horizontal line equation ax + by + c = 0 
            which satisfies a**2 + b**2 = 1
        pt1 (optional): np.ndarray of shape (3, )
            where one horizontal 2d vanishing point in homogeneous coordinate
        pt2 (optional): np.ndarray of shape (3, )
            where the other horizontal 2d vanishing point in homogeneous coordinate
    Returns:
    """
    # normalize horizontal_line
    horizontal_line /= np.linalg.norm(horizontal_line[:-1])

    ax.imshow(im)

    if (pt1 is None) or (pt2 is None): 
        # if two horizontal vanishing points are not given, 
        # pt1 for x-intercept and pt2 for y-intercept
        pt1 = np.array([-horizontal_line[2]/horizontal_line[0], 0, 1])
        pt2 = np.array([0, -horizontal_line[2]/horizontal_line[1], 1])
    else:
        # else, normalize them
        pt1 /= pt1[-1]
        pt2 /= pt2[-1]

    # plot horizontal line
    ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    # plot vanishing points or intercepts
    ax.plot(pt1[0], pt1[1], 'ro')
    ax.plot(pt2[0], pt2[1], 'ro')

In [8]:
def get_camera_parameters(vpt_x, vpt_y, vpt_z):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    Inputs:
        vpt_x: np.ndarray of shape (3, )
            where horizontal vanishing point toward right in 2d homogeneous coordinate
        vpt_y: np.ndarray of shape (3, )
            where vertical vanishing point downward in 2d homogeneous coordinate
        vpt_z: np.ndarray of shape (3, )
            where horizontal vanishing point toward left in 2d homogeneous coordinate
    Returns:
        f: focal length
        u: principal point along horizontal axis
        v: principal point along vertical axis
        K: np.ndarray of shape (3, 3)
    """
    from sympy import symbols, Matrix, solve, Eq
    f_sym, u_sym, v_sym = symbols('f, u, v')
    K_sym = Matrix([[f_sym,0,u_sym], [0,f_sym,v_sym], [0,0,1]])

    V_left  = np.vstack([vpt_x, vpt_y, vpt_z]).T
    V_right = np.vstack([vpt_y, vpt_z, vpt_x]).T

    res = V_left.T * K_sym.inv().T * K_sym.inv() * V_right
    
    eq0 = Eq(res[0,0], 0)
    eq1 = Eq(res[1,1], 0)
    eq2 = Eq(res[2,2], 0)

    sol = solve([eq0, eq1, eq2], (f_sym, u_sym, v_sym))
    f = abs(sol[0][0])
    u = sol[0][1]
    v = sol[0][2]
    K = np.array([[f,0,u], [0,f,v], [0,0,1]], dtype='float')

    return f, u, v, K

In [9]:
def get_rotation_matrix(vpt_x, vpt_y, vpt_z, K):
    """
    Computes the rotation matrix using the camera parameters.
    Inputs:
        vpt_x: np.ndarray of shape (3, )
            where horizontal vanishing point toward right in 2d homogeneous coordinate
        vpt_y: np.ndarray of shape (3, )
            where vertical vanishing point downward in 2d homogeneous coordinate
        vpt_z: np.ndarray of shape (3, )
            where horizontal vanishing point toward left in 2d homogeneous coordinate
        K: np.ndarray of shape (3, 3)
    Returns:
        R: np.ndarray of shape (3, 3)
            rotation matrix, whose column vectors' norms are equal to 1
    """
    # calculate unnormalized rotation matrix
    r1_raw = np.linalg.inv(K) @ vpt_x[:,np.newaxis]
    r2_raw = np.linalg.inv(K) @ vpt_y[:,np.newaxis]
    r3_raw = np.linalg.inv(K) @ vpt_z[:,np.newaxis]
    
    # normalize columnwise
    r1 = r1_raw / np.linalg.norm(r1_raw)
    r2 = r2_raw / np.linalg.norm(r2_raw)
    r3 = r3_raw / np.linalg.norm(r3_raw)

    R = np.hstack([r1,r2,r3])

    return R

# Main function

In [10]:
im = np.asarray(Image.open('./eceb.jpg'))

# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('./data.pickle'):
    with open('./data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f)
    num_vpts = 3

In [11]:
# Click and save the line data for vanishing points. This snippet 
# opens up an interface for selecting points and writes them to 
# data.pickle file. The file is over-written.

# num_vpts = 3
# all_n, all_lines, all_centers = [], [], []
# 
# for i in range(num_vpts):
#     print(f'Getting vanishing point {i}')
#     
#     # Get at least three lines from user input
#     n_i, lines_i, centers_i = get_input_lines(im)
#     all_n.append(n_i)
#     all_lines.append(lines_i)
#     all_centers.append(centers_i)
# 
# with open('data.pickle', 'wb') as f:
#     pickle.dump([all_n, all_lines, all_centers], f)

In [12]:
# Part (1)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))

for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    # <YOUR CODE> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(all_lines[i])
    print(f"vanishing point {i}: ", vpts[:2, i])
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    fig.savefig('Q3_vp{:d}.pdf'.format(i), bbox_inches='tight')
    fig.savefig('Q3_vp{:d}.jpg'.format(i))

vanishing point 0:  [-1910.80224301    93.81311665]
vanishing point 1:  [3679.66328067 -105.84809264]
vanishing point 2:  [2175.82617724 5636.70907416]


In [13]:
# Part (2) Computing and plotting the horizon
# <YOUR CODE> Get the ground horizon line
horizon_line = get_horizon_line(vpts[:, 0], vpts[:, 1])
print("horizontal line: ", horizon_line)

# <YOUR CODE> Plot the ground horizon line
fig = plt.figure(); ax = fig.gca()
plot_horizon_line(ax, im, horizon_line, pt1=vpts[:, 0], pt2=vpts[:, 1])
fig.savefig('Q3_horizon.pdf', bbox_inches='tight')
fig.savefig('Q3_horizon.jpg')

horizontal line:  [-0.03569184 -0.99936284 25.55329752]


In [14]:
# Part (3) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
f, u, v, K = get_camera_parameters(vpt_x=vpts[:,1], vpt_y=vpts[:,2], vpt_z=vpts[:,0])
print(f'f: {f:.2f}, u: {u:.2f}, v: {v:.2f}')
print('K: \n', K)

# sanity check for whether the estimated intrinsic parameters are valid; the matrix multiplication should be minimized as much as possible
K_invT_K_inv = np.array([[f**(-2), 0, -u/f**2], 
                          [0, f**(-2), -v/f**2],
                          [-u/f**2, -v/f**2, 1 + u**2/f**2 + v**2/f**2]])
print('test 1: ', vpts[:,0][np.newaxis, :] @ K_invT_K_inv @ vpts[:,1])
print('test 2: ', vpts[:,1][np.newaxis, :] @ K_invT_K_inv @ vpts[:,2])
print('test 3: ', vpts[:,2][np.newaxis, :] @ K_invT_K_inv @ vpts[:,0])

f: 2296.55, u: 2014.58, v: 1121.78
K: 
 [[2.29655057e+03 0.00000000e+00 2.01457715e+03]
 [0.00000000e+00 2.29655057e+03 1.12177522e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
test 1:  [2.66453525910038e-15]
test 2:  [8.88178419700125e-16]
test 3:  [-1.63064006741820e-15]


In [15]:
# Part (4) Computing Rotation Matrices
# <YOUR CODE> Solve for the rotation matrix
R = get_rotation_matrix(vpt_x=vpts[:,1], vpt_y=vpts[:,2], vpt_z=vpts[:,0], K=K)
print('R: \n', R)

# sanity check for whether R is orthonormal
def isRotationMatrix(M, eps=0.01):
    I = np.identity(M.shape[0])
    if np.all(np.around(M @ M.T) == I) and (np.linalg.det(M)-1 < eps): 
        print('R is a rotation matrix.')
        return True
    else: 
        print('R is not a rotation matrix.')
        return False

isRotationMatrix(R)

R: 
 [[ 0.53870444  0.031817   -0.84189382]
 [-0.39717232  0.89086833 -0.22047167]
 [ 0.7430018   0.45314599  0.49255156]]
R is a rotation matrix.


True